In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel_transfer, find_state, directory_scratch, directory_project, directory_archive, run_parallel_transfer_to_archive
import dropbox
from amftrack.transfer.functions.transfer import upload
from subprocess import call

/home/cbisot/pycode/MscThesis/amftrack/transfer/functions/transfer.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# directory = directory_project
# directory = '/run/user/357100554/gvfs/smb-share:server=sun.amolf.nl,share=shimizu-data,user=bisot/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/'
directory = "dropbox"
# directory = '/projects/0/einf914/agg/'

# update_plate_info("directory")

In [4]:

%autoreload 2
all_folders = get_current_folders(directory)
folders = all_folders.loc[all_folders['Plate']==737]

analysed:   0%|          | 0/692 [00:00<?, ?it/s]

In [5]:
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,unique_id
15,/run/user/357100554/gvfs/smb-share:server=sun....,737,34,Carrot,Agg,100P100N,Y,-,20210901,False,...,False,False,False,False,False,False,"19.09.2021, 18:09:",20210919_1809_Plate34,NaN,73720210901
24,/run/user/357100554/gvfs/smb-share:server=sun....,737,34,Carrot,Agg,100P100N,Y,-,20210901,False,...,False,False,False,False,False,False,"08.09.2021, 13:02:",20210908_1302_Plate34,NaN,73720210901
62,/run/user/357100554/gvfs/smb-share:server=sun....,737,34,Carrot,Agg,100P100N,Y,-,20210901,False,...,False,False,False,False,False,False,"07.09.2021, 19:48:",20210907_1948_Plate34,NaN,73720210901
76,/run/user/357100554/gvfs/smb-share:server=sun....,737,34,Carrot,Agg,100P100N,Y,-,20210901,False,...,False,False,False,False,False,False,"11.09.2021, 04:58:",20210911_0458_Plate34,NaN,73720210901
92,/run/user/357100554/gvfs/smb-share:server=sun....,737,34,Carrot,Agg,100P100N,Y,-,20210901,False,...,False,False,False,False,False,False,"05.09.2021, 17:41:",20210905_1741_Plate34,NaN,73720210901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5323,/run/user/357100554/gvfs/smb-share:server=sun....,737,34,Carrot,Agg,100P100N,Y,-,20210901,False,...,False,False,False,False,False,False,"07.09.2021, 09:40:",20210907_0940_Plate34,NaN,73720210901
5330,/run/user/357100554/gvfs/smb-share:server=sun....,737,34,Carrot,Agg,100P100N,Y,-,20210901,False,...,False,False,False,False,False,False,"16.09.2021, 00:01:",20210916_0001_Plate34,NaN,73720210901
5341,/run/user/357100554/gvfs/smb-share:server=sun....,737,34,Carrot,Agg,100P100N,Y,-,20210901,False,...,False,False,False,False,False,False,"07.09.2021, 21:48:",20210907_2148_Plate34,NaN,73720210901
5351,/run/user/357100554/gvfs/smb-share:server=sun....,737,34,Carrot,Agg,100P100N,Y,-,20210901,False,...,False,False,False,False,False,False,"24.09.2021, 12:00:",20210924_1200_Plate34,NaN,73720210901


In [36]:
directory_targ = os.path.join(directory_scratch,'temp')+'/'
# directory_targ = os.path.join(directory_archive,'Prince_Data')+'/'
directory_targ = '/projects/0/einf914/agg/'


**To transfer data from surfsara to dropbox**
- use the 'toward_drop.py' function including the folders of interest and using dir_drop as a parameter indicating where in dropbox the data should go


In [7]:
dir_drop = 'prince_data'
run_parallel_transfer('toward_drop.py',[directory,dir_drop],folders,5,'20:00','staging',cpus = 1,node = 'staging')

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 248354
Submitted batch job 248355
Submitted batch job 248356
Submitted batch job 248357


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 248358
Submitted batch job 248359
Submitted batch job 248360
Submitted batch job 248361
Submitted batch job 248362
Submitted batch job 248363
Submitted batch job 248364
Submitted batch job 248365
Submitted batch job 248366


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a n

Submitted batch job 248372
Submitted batch job 248373
Submitted batch job 248374
Submitted batch job 248375
Submitted batch job 248376


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 248377
Submitted batch job 248378
Submitted batch job 248379
Submitted batch job 248380
Submitted batch job 248381
Submitted batch job 248382
Submitted batch job 248383


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


**To transfer data from dropbox to surfsara**
- use the 'from_drop.py' function including the folders of interest
- unzip argument decides if folders must also be unzipped
- flatten argument decides if folders must be flatten to respect snellius paralell computing logic or kept in a folder for transfer to archive for example

In [37]:
dir_drop = 'prince_data'
unzip = True
flatten = True
run_parallel_transfer('from_drop.py',[directory_targ,dir_drop,unzip,flatten],folders,5,'10:00','staging',cpus = 1,node = 'staging')

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 275403
Submitted batch job 275404
Submitted batch job 275405


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 275406
Submitted batch job 275407
Submitted batch job 275408
Submitted batch job 275409
Submitted batch job 275411
Submitted batch job 275412


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 275443
Submitted batch job 275444
Submitted batch job 275445


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 275446
Submitted batch job 275447
Submitted batch job 275448
Submitted batch job 275449
Submitted batch job 275450
Submitted batch job 275451


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [34]:
from amftrack.transfer.functions.transfer import download, zip_file,unzip_file

unzip_file('/scratch-shared/amftrack/temp/20210901_2139_Plate34.zip','/projects/0/einf914/data/agg/20210901_2139_Plate34')

**To transfer data from dropbox to archive**
- use the 'from_drop.py' function including the folders of interest


In [4]:
folders = all_folders.loc[all_folders['Plate'].isin([792,94])]

In [6]:
unzip = 'False'
flatten = False
dir_drop = 'prince_data'

directory_targ = os.path.join(directory_scratch,'temp')+'/'
directory = directory_targ
run_parallel_transfer('from_drop.py',[directory_targ,dir_drop,unzip,flatten],folders,5,'10:00','staging',cpus = 1,node = 'staging',name_job='archiving.sh')
run_parallel_transfer_to_archive(folders,directory,'10:00','staging',name_job='archiving.sh')

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 298787
Submitted batch job 298788
Submitted batch job 298789
Submitted batch job 298790
Submitted batch job 298791
Submitted batch job 298792


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 298793
Submitted batch job 298794
Submitted batch job 298795
Submitted batch job 298796
Submitted batch job 298797


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 298826
Submitted batch job 298827
Submitted batch job 298828
Submitted batch job 298829
Submitted batch job 298830


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 298831
Submitted batch job 298832
Submitted batch job 298833
Submitted batch job 298834
Submitted batch job 298835


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [11]:
run_parallel_transfer_to_archive(folders,directory,'10:00','staging',name_job='archiving.sh')

Submitted batch job 271711
Submitted batch job 271712


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [12]:
must_zip = True
must_unzip = False
run_parallel_transfer('within_surf.py',[directory,directory_targ,must_zip,must_unzip],folders,5,'10:00','staging',cpus = 1,node = 'staging')

Submitted batch job 242022


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [7]:
plates = set(folders['Plate'].values)

In [10]:
call(f"tar cvf /archive/cbisot/prince_data/94_20201123.tar {directory_targ}94_20201123*", shell=True)

tar: /archive/cbisot/prince_data/94_20201123.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


2

In [ ]:
def transfer_parallel_whole_plate(code, args, folders, num_parallel, time, name,cpus = 128,node = 'thin'):
    op_id = time_ns()
    folders.to_json(f'{directory_scratch}temp/{op_id}.json')# temporary file
    plates = set(folders['Plate'].values)
    length = len(plates)
    begin_skel = 0
    end_skel = length // num_parallel + 1
    args_str = [str(arg) for arg in args]
    arg_str = " ".join(args_str)
    arg_str_out = "_".join([str(arg) for arg in args if type(arg)!=str])
    for j in range(begin_skel, end_skel):
        start = num_parallel * j
        stop = num_parallel * j + num_parallel - 1
        ide = time_ns()
        my_file = open(path_job, "w")
        my_file.write(
            f"#!/bin/bash \n#Set job requirements \n#SBATCH --nodes=1 \n#SBATCH -t {time}\n #SBATCH --ntask=1 \n#SBATCH --cpus-per-task={cpus}\n#SBATCH -p {node} \n"
        )
        my_file.write(
            f'#SBATCH -o "{path_code}slurm/{name}_{arg_str_out}_{start}_{stop}_{ide}.out" \n'
        )
        my_file.write(f"source /home/cbisot/miniconda3/etc/profile.d/conda.sh\n")
        my_file.write(f"conda activate amftrack\n")
        my_file.write(f"for i in `seq {start} {stop}`; do\n")
        my_file.write(f"\t python {path_code}amftrack/pipeline/scripts/image_processing/{code} {arg_str} {op_id} $i &\n")
        my_file.write("done\n")
        my_file.write("wait\n")
        my_file.close()
        call(f"sbatch {path_job}", shell=True)

In [8]:
def run_parallel_transfer(code, args, folders, num_parallel, time, name,cpus = 1,node = 'staging',name_job='transfer.sh'):
    path_job = f'{path_bash}{name_job}'
    op_id = time_ns()
    folders.to_json(f'{directory_scratch}temp/{op_id}.json')# temporary file
    length = len(folders)
    begin_skel = 0
    end_skel = length // num_parallel + 1
    args_str = [str(arg) for arg in args]
    arg_str = " ".join(args_str)
    arg_str_out = "_".join([str(arg) for arg in args if type(arg)!=str])
    for j in range(begin_skel, end_skel):
        start = num_parallel * j
        stop = num_parallel * j + num_parallel - 1
        ide = time_ns()
        my_file = open(path_job, "w")
        my_file.write(
            f"#!/bin/bash \n#Set job requirements \n#SBATCH --nodes=1 \n#SBATCH -t {time}\n #SBATCH --ntask=1 \n#SBATCH --cpus-per-task={cpus}\n#SBATCH -p {node} \n"
        )
        my_file.write(
            f'#SBATCH -o "{path_code}slurm/{name}_{arg_str_out}_{start}_{stop}_{ide}.out" \n'
        )
        my_file.write(f"source /home/cbisot/miniconda3/etc/profile.d/conda.sh\n")
        my_file.write(f"conda activate amftrack\n")
        my_file.write(f"for i in `seq {start} {stop}`; do\n")
        my_file.write(f"\t python {path_code}amftrack/transfer/scripts/{code} {arg_str} {op_id} $i &\n")
        my_file.write("done\n")
        my_file.write("wait\n")
        my_file.close()
        call(f"sbatch {path_job}", shell=True)